In [3]:
import pandas as pd
from IPython.display import Image, HTML
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
listings = pd.read_csv('list.csv', usecols = ['id', 'product', 'description', 'category', 'sub_category'])
listings.head(10)

,id,product,description,sub_category,category
0,241032,Stylish Queen Anne Apartment,Make your self at home in this charming one-be...,Seattle,WA
1,953595,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,Seattle,WA
2,3308979,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,Seattle,WA
3,7421966,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,Seattle,WA
4,278830,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Seattle,WA
5,5956968,Private unit in a 1920s mansion,We're renting out a small private unit of one ...,Seattle,WA
6,1909058,Queen Anne Private Bed and Bath,Enjoy a quiet stay in our comfortable 1915 Cra...,Seattle,WA
7,856550,Tiny Garden cabin on Queen Anne,"Our tiny cabin is private , very quiet and com...",Seattle,WA
8,4948745,Urban Charm || Downtown || Views,"Nestled in the heart of the city, this space i...",Seattle,WA
9,2493658,Airy + Bright Queen Anne Apartment,"Beautiful apartment in an extremely safe, quie...",Seattle,WA


In [6]:
listings['product'] = listings['product'].astype('str')
listings['description'] = listings['description'].astype('str')

In [7]:


name_corpus = ' '.join(listings['product'])
description_corpus = ' '.join(listings['description'])





name_wordcloud = WordCloud(stopwords = STOPWORDS, background_color = 'white', height = 2000, width = 4000).generate(name_corpus)
plt.figure(figsize = (16,8))
plt.imshow(name_wordcloud)
plt.axis('off')
plt.show()




description_wordcloud = WordCloud(stopwords = STOPWORDS, background_color = 'white', height = 2000, width = 4000).generate(description_corpus)
plt.figure(figsize = (16,8))
plt.imshow(description_wordcloud)
plt.axis('off')
plt.show()

In [8]:
listings['content'] = listings[['product', 'description']].astype(str).apply(lambda x: ' // '.join(x), axis = 1)

In [9]:
listings['content'].fillna('Null', inplace = True)

In [10]:


tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(listings['content'])



In [11]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [12]:


results = {}
for idx, row in listings.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], listings['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]



In [13]:
def item(id):
    product   = listings.loc[listings['id'] == id]['content'].tolist()[0].split(' // ')[0]
    desc   = ' \nDescription: ' + listings.loc[listings['id'] == id]['content'].tolist()[0].split(' // ')[1][0:165] + '...'
    prediction = product  + desc
    return prediction

def recommend(product_id, num):
    print('Recommending ' + str(num) + ' products similar to ' + item(product_id))
    print('---')
    recs = results[product_id][:num]
    for rec in recs:
        print('\nRecommended: ' + item(rec[1]) + '\n(score:' + str(rec[0]) + ')')

In [25]:
recommend(product_id = 856550, num = 1)

Recommending 1 products similar to Tiny Garden cabin on Queen Anne   
Description: Our tiny cabin is private , very quiet and comfortable spot just for two. The bus is just across the street. Queen Anne ,Ballard and Downtown are a short bus ride. H...
---
63    \nRecommended: Seattle\n(score:0.1220372829053...
Name: sub_category, dtype: object


In [15]:
def item(id):
    name   = listings.loc[listings['id'] == id]['content'].tolist()[0].split(' // ')[0]
    desc   = ' \nDescription: ' + listings.loc[listings['id'] == id]['content'].tolist()[0].split(' // ')[1][0:165] + '...'
    prediction = name  + desc
    return prediction

def product_id(ide):
    prediction = ide.astype(str)
    return prediction

def recommend(product_id, num):
    print('Recommending ' + str(num) + ' products similar to ' + item(product_id))
    print('---')
    recs = results[product_id][:num]
    for rec in recs:
        print('\nRecommended: ' + subcat(rec[1]) + '\n(score:' + str(rec[0]) + ')')

In [23]:
def item(id):
    name   = listings.loc[listings['id'] == id]['content'].tolist()[0].split(' // ')[0]
    desc   = ' \nDescription: ' + listings.loc[listings['id'] == id]['content'].tolist()[0].split(' // ')[1][0:165] + '...'
    prediction = name  + desc
    return prediction

def product_(id):
    prediction = listings[listings['id']==id]['product']
    return prediction

def recommend(product_id, num):
    print('Recommending ' + str(num) + ' products similar to ' + item(product_id))
    print('---')
    recs = results[product_id][:num]
    for rec in recs:
        print('\nRecommended: ' + subcat(rec[1]) + '\n(score:' + str(rec[0]) + ')')

In [17]:
import pickle
pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))
pickle.dump(selector, open("selector.pickle", "wb"))

NameError: name 'vectorizer' is not defined